In [1]:
import autokeras
import pandas
import numpy

In [2]:
n = 1000
df = pandas.DataFrame(
    {
        "x": numpy.random.rand(n),
        "y": numpy.random.rand(n)
    }
)
df["z"] = df["x"].apply(lambda x: 1 if x >= 0.5 else 0)
df_train = df[0:800]
df_test = df[800:]

x_train = numpy.array(df_train[["x", "y"]])
y_train = numpy.array(df_train["z"])

x_test = numpy.array(df_test[["x", "y"]])
y_test = numpy.array(df_test["z"])

In [3]:
df.head(5)

,x,y,z
0,0.488413,0.856357,0
1,0.708752,0.219288,1
2,0.885611,0.555843,1
3,0.828803,0.956945,1
4,0.860225,0.744252,1


In [4]:
model = autokeras.ImageClassifier()

In [6]:
model.fit(x_train.reshape(800, 2, 1), y_train, time_limit=0.05*60*60)

In [7]:
model.final_fit(
    x_train.reshape(800, 2, 1),
    y_train,
    x_test.reshape(200, 2, 1),
    y_test,
    retrain=False)

In [8]:
model.predict(numpy.array([0.6, 0.1]).reshape(1,2,1))

array([1], dtype=int64)

In [9]:
model.predict(numpy.array([0.4, 0.1]).reshape(1,2,1))

array([0], dtype=int64)

In [10]:
model.predict(numpy.array([[0.45, 0.1], [0.55, 0.8]]).reshape(2,2,1))

array([0, 1], dtype=int64)

In [11]:
model.cnn.searcher.history

[{'model_id': 0,
  'loss': 0.05906128436326981,
  'metric_value': 0.9727272727272727},
 {'model_id': 1,
  'loss': 0.07171969339251519,
  'metric_value': 0.9575757575757576}]

In [12]:
model.cnn.best_model.produce_model()

TorchModel(
  (0): ReLU()
  (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): ReLU()
  (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (7): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): ReLU()
  (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (11): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): GlobalAvgPool1d()
  (13): Dropout(p=0.25)
  (14): Linear(in_features=64, out_features=64, bias=True)
  (15): ReLU()
  (16): Linear(in_features=64, out_features=2, bias=True)
)

In [13]:
model.cnn.best_model.layer_list

In [14]:
path = "model.pkl"
model.export_autokeras_model(path)

In [15]:
# 予測

In [16]:
output_model = autokeras.utils.pickle_from_file(path)

In [17]:
output_model.predict(numpy.array([[0.45, 0.1], [0.55, 0.8], [0.49, 0.1]]).reshape(3,2,1))

array([0, 1, 0], dtype=int64)